# Prepare docker

In [ ]:
# !docker build -t elastiknn-semantic-search .        

In [1]:
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [2]:
!docker-compose ps

NAME                   COMMAND                  SERVICE                STATUS              PORTS
elasticsearch_master   "/bin/tini -- /usr/lвЂ¦"   elasticsearch_master   exited (143)        


In [3]:
# !docker run -d -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" --name search-server -v elasticsearch.yml:/usr/share/elasticsearch/config/elasticsearch.yml elastiknn-semantic-search

In [ ]:
!docker-compose up -d
# !docker-compose start semantic-search

In [ ]:
!curl --silent -u elastic:11235813 localhost:9200
!curl --silent -u elastic:11235813 localhost:9200/_cluster/stats?pretty
!curl --silent -u elastic:11235813 localhost:9200/_cat/health
!curl --silent -u elastic:11235813 localhost:9200/_cat/plugins

In [33]:
# !poetry add elasticsearch==7.17.3
# !poetry add tqdm

Using version ^4.64.0 for tqdm

Updating dependencies
Resolving dependencies...

Writing lock file

Package operations: 1 install, 0 updates, 0 removals

  • Installing tqdm (4.64.0)


In [ ]:
# !curl -X POST "localhost:9200/_nodes/reload_secure_settings?pretty" -H 'Content-Type: application/json' -d' \
# { \
#   "secure_settings_password": "keystore-password" \
# } \
# ' \


In [23]:
!pip list | grep elasticsearch 

elasticsearch           7.17.3


You should consider upgrading via the 'C:\Users\artif\AppData\Local\pypoetry\Cache\virtualenvs\semantic-search-5CWfmEyu-py3.8\Scripts\python.exe -m pip install --upgrade pip' command.


# Setup Elasticsearch client

In [63]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

es = Elasticsearch(["http://localhost:9200"], http_auth=('elastic', '11235813'), timeout=30)
es.cluster.health(wait_for_status='yellow', request_timeout=10)


{'cluster_name': 'docker-cluster',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 1793,
 'active_shards': 1793,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 1789,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 50.055834729201564}

### check client api version

In [9]:
import elasticsearch as eees
eees.__version__

(7, 17, 3)

## Prepare indices

In [87]:
# delete all indices
# !curl -s -X DELETE localhost:9200/_all


{"acknowledged":true}


In [88]:
index_name = 'news-headers'

source_no_vecs = ['title', 'db_id']
vector_dims = 512

In [89]:
# es.indices.get(index="*")
es.indices.get(index='*')

{}

### optional: delete index

In [91]:
# es.indices.delete(index=index_name, ignore=[400, 404])
# es.indices.flush(index="*")
# es.indices.shrink(index="*")
# if es.indices.exists(index=index_name):
# es.indices.get_mapping(index=index_name)

In [92]:
settings = {
    "number_of_shards": 1,
    "number_of_replicas": 0,
    "elastiknn": True,    
  # "settings": {
  #   "number_of_shards": 1,
  #   "number_of_replicas": 0,
  #   "elastiknn": True,    
  # }
}

mapping = {
  "dynamic": False,
  "properties": {
    "db_id": { "type": "text" },
    "title": {"type": "text" },  
      
      
    "title_emb": {
        "type": "elastiknn_dense_float_vector", # 1
        "elastiknn": {
            "dims": vector_dims,                        # 2
            "model": "lsh",                     # 3
            "similarity": "cosine",             # 4
            "L": 99,                            # 5
            "k": 3                              # 6
        }
    },
    
    # "imVecElastiknn": {
    #   "type": "elastiknn_dense_float_vector",
    #   "elastiknn": {
    #     "dims": vector_dims,
    #     "model": "lsh",
    #     "similarity": "angular",
    #     "L": 60,
    #     "k": 3
    #   }
    # },
    # "imVecXpack": {
    #   "type": "dense_vector",
    #   "dims": vector_dims
    # },
    # "title": { "type": "text" },
    # "description": { "type": "text" },
    # "price": { "type": "float" },
    # "imUrl": { "type": "text" }
  }
}

if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, settings=settings)
    es.indices.put_mapping(mapping, index=index_name,  )


C:\Users\artif\AppData\Local\Temp\ipykernel_11120\1924555001.py:53: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.put_mapping(mapping, index=index_name,  )


In [93]:
es.cluster.put_settings({
    "persistent": { "cluster.max_shards_per_node": "5000" },
    "transient": {
        "cluster.routing.allocation.total_shards_per_node": 5100
    }
})

C:\Users\artif\AppData\Local\Temp\ipykernel_11120\3899948553.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.cluster.put_settings({


{'acknowledged': True,
 'persistent': {'cluster': {'max_shards_per_node': '5000'}},
 'transient': {'cluster': {'routing': {'allocation': {'total_shards_per_node': '5100'}}}}}

In [94]:
es.indices.get_settings(index=index_name)

{'news-headers': {'settings': {'index': {'elastiknn': 'true',
    'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'number_of_shards': '1',
    'provided_name': 'news-headers',
    'creation_date': '1653891551392',
    'number_of_replicas': '0',
    'uuid': '9NuFNuRrSXmb_u6purdjIA',
    'version': {'created': '7170399'}}}}}

In [95]:
es.indices.refresh(index=index_name)

{'_shards': {'total': 1, 'successful': 1, 'failed': 0}}

## Generate and index Embeddings

In [96]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
from tqdm import tqdm
import pandas as pd

In [97]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [98]:
tqdm.pandas(desc="pandas processing...")
df = pd.read_excel("ЭТ_Новости.xlsx")
df.head()

,ID,NEWS_HEAD
0,1552621575,Государственная Дума рекомендует Правительству...
1,1552115547,Евгений Грабчак: «Меры поддержки помогут ТЭК о...
2,1552115549,Павел Сниккарс: «Важная задача в электроэнерге...
3,1551704812,Вниманию гарантирующих поставщиков и энергосбы...
4,1551704814,Эдуард Шереметцев: «Национальная энергетическа...


In [99]:
def heads_actions():
    for index, row in df.iterrows():
        # print(row['ID'], row['NEWS_HEAD'])
        # print(embed(row['NEWS_HEAD']).numpy()[0])
        yield {
          "_op_type": "index", 
          "_index": index_name, 
            
          "_id": row['ID'], 
          "db_id": row['ID'],      
          "title": row['NEWS_HEAD'] 
          # "description": p.get("description", None),
          # "price": p.get("price", None),
          # "imUrl": p.get("imUrl", None)
        }

bulk(es, heads_actions(), chunk_size=1000, max_retries=2)
        

(1975, [])

In [70]:

# for i, row in df.iterrows():
#     doc = {
#           "db_id": row['ID'],      
#           "title": row['NEWS_HEAD'] 
#         }
#     res = es.index(index=index_name, id=row['ID'], body=doc)
#     print(res)

C:\Users\artif\AppData\Local\Temp\ipykernel_11120\982036466.py:6: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.index(index=index, id=row['ID'], body=doc)


In [100]:
es.indices.refresh(index=index_name)
es.indices.forcemerge(index=index_name, max_num_segments=1, request_timeout=120)

{'_shards': {'total': 1, 'successful': 1, 'failed': 0}}

In [108]:
def vector_actions():
    for index, row in df.iterrows():
        yield { 
            "_op_type": "update", 
            "_index": index_name, 
            
            "_id": row['ID'], 
            "doc": { 
              "title_emb": { "values": embed(row['NEWS_HEAD']).numpy()[0] }
            }
        }
        

bulk(es, vector_actions(), chunk_size=50, max_retries=10, request_timeout=60)

(1975, [])

In [121]:
es.indices.refresh(index=index_name)
es.indices.forcemerge(index=index_name, max_num_segments=1, request_timeout=300)

{'_shards': {'total': 1, 'successful': 1, 'failed': 0}}

In [110]:
es.indices.get(index_name)

C:\Users\artif\AppData\Local\Temp\ipykernel_11120\1695063147.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.get(index_name)


{'news-headers': {'aliases': {},
  'mappings': {'dynamic': 'false',
   'properties': {'db_id': {'type': 'text'},
    'title': {'type': 'text'},
    'title_emb': {'type': 'elastiknn_dense_float_vector',
     'elastiknn': {'L': 99,
      'dims': 512,
      'k': 3,
      'model': 'lsh',
      'similarity': 'cosine'}}}},
  'settings': {'index': {'elastiknn': 'true',
    'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'number_of_shards': '1',
    'provided_name': 'news-headers',
    'creation_date': '1653891551392',
    'number_of_replicas': '0',
    'uuid': '9NuFNuRrSXmb_u6purdjIA',
    'version': {'created': '7170399'}}}}}

### Check documents in index

In [115]:
def display_hits(res):
    print(f"Found {res['hits']['total']['value']} hits in {res['took']} ms. Showing top {len(res['hits']['hits'])}.")
    print("")
    for hit in res['hits']['hits']:
        s = hit['_source']
        print(f"Id:{s.get('db_id', None)} Title   {s.get('title', None)}")
        print(f"Score   {hit.get('_score', None)}")


In [ ]:
es.get(index=index_name, id="1552115547")

# Keyword search

In [116]:
body = {
  "query": {
    "multi_match": {
      "query": "Грабчак",
      "fields": ["title^2"]
    }
  }
}

res = es.search(index=index_name, body=body, size=5, _source=source_no_vecs)
display_hits(res)

Found 21 hits in 6 ms. Showing top 5.

Id:1103327850 Title   Евгений Грабчак: «Министерство энергетики поддерживает консолидацию электросетевого комплекса»
Score   10.686453
Id:1100853084 Title   Евгений Грабчак принял участие в Конференции по киберзащите промышленных объектов
Score   9.937215
Id:1100853110 Title   Евгений Грабчак открыл Всероссийские соревнования профессионального мастерства оперативного персонала ТЭС
Score   9.937215
Id:1100853360 Title   Евгений Грабчак: «Необходимо внедрять технологии цифрового моделирования в отраслевую деятельность»
Score   9.937215
Id:1100853397 Title   Евгений Грабчак принял участие в конференции экономико-экологического форума ОБСЕ
Score   9.937215


C:\Users\artif\AppData\Local\Temp\ipykernel_11120\1281547386.py:10: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index=index_name, body=body, size=5, _source=source_no_vecs)


# Semantic search

In [ ]:
db_id = 1103327850

fetch_res = es.get(index=index_name, id=db_id)
query_vec = fetch_res['_source']['title_emb']['values']
query_vec

In [126]:
body = {
    "query": {
        "elastiknn_nearest_neighbors": {
            "field": "title_emb",                     # 1
            "vec": {                               # 2
                "values": query_vec
            },
            "model": "lsh",                        # 3
            "similarity": "cosine",                # 4
            "candidates": 50                       # 5
        }
    }
}

res = es.search(index=index_name, body=body, size=5, _source=source_no_vecs)
display_hits(res)

Found 50 hits in 147 ms. Showing top 5.

Id:1103327850 Title   Евгений Грабчак: «Министерство энергетики поддерживает консолидацию электросетевого комплекса»
Score   2.0
Id:1100853086 Title   Юрий Маневич: «Министерство энергетики выступает за конкуренцию на рынке электроэнергии»
Score   1.6889415
Id:1552115547 Title   Евгений Грабчак: «Меры поддержки помогут ТЭК обеспечить российскую экономику необходимыми энергоресурсами» 
Score   1.6439109
Id:1547982596 Title   Евгений Грабчак: «Платежи легальных майнеров за электроэнергию могут обеспечить дополнительные средства для развития электросетей» 
Score   1.64228
Id:1110168307 Title   Евгений Грабчак: «Цифровая модернизация позволит решить высокий уровень износа основных фондов в электроэнергетике»
Score   1.6400468


C:\Users\artif\AppData\Local\Temp\ipykernel_11120\1840879591.py:15: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index=index_name, body=body, size=5, _source=source_no_vecs)


In [130]:
query_text = "события в области"
query_vec = embed(query_text).numpy()[0]

body = {
    "query": {
        "elastiknn_nearest_neighbors": {
            "field": "title_emb",                     # 1
            "vec": {                               # 2
                "values": query_vec
            },
            "model": "lsh",                        # 3
            "similarity": "cosine",                # 4
            "candidates": 150                       # 5
        }
    }
}

res = es.search(index=index_name, body=body, size=10, _source=source_no_vecs)
display_hits(res)

Found 150 hits in 9 ms. Showing top 10.

Id:1100853659 Title    О мероприятиях в Тамбовской области
Score   1.4850954
Id:1100853593 Title   Новости Амурской области
Score   1.4110494
Id:1100853785 Title   Новости Московской области
Score   1.4084431
Id:1102612044 Title   Новости Тульской области 
Score   1.4013536
Id:1102253414 Title   Новости Тульской области
Score   1.4013536
Id:1100853647 Title    Новости Орловской области
Score   1.3985813
Id:1100851835 Title   Новости Курганской области
Score   1.3968015
Id:1100853629 Title    Новости Курганской области
Score   1.3968015
Id:1113175798 Title   Новости Кемеровской области
Score   1.3895586
Id:1100853759 Title   О мероприятии в Аппарате Рязанской областной Думы
Score   1.385992


C:\Users\artif\AppData\Local\Temp\ipykernel_11120\516059528.py:18: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index=index_name, body=body, size=10, _source=source_no_vecs)


# deprecated: some curl tests

In [24]:
!curl -s -XGET localhost:9200/_cluster/allocation/explain?pretty

{
  "note" : "No shard was specified in the explain API request, so this response explains a randomly chosen unassigned shard. There may be other unassigned shards in this cluster which cannot be assigned for different reasons. It may not be possible to assign this shard until one of the other shards is assigned correctly. To explain the allocation of other shards (whether assigned or unassigned) you must specify the target shard in the request to this API.",
  "index" : "1505",
  "shard" : 0,
  "primary" : false,
  "current_state" : "unassigned",
  "unassigned_info" : {
    "reason" : "CLUSTER_RECOVERED",
    "at" : "2022-05-30T04:47:33.638Z",
    "last_allocation_status" : "no_attempt"
  },
  "can_allocate" : "no",
  "allocate_explanation" : "cannot allocate because allocation is not permitted to any of the nodes",
  "node_allocation_decisions" : [
    {
      "node_id" : "_cGWOoLjRqSIXGg95H_VDw",
      "node_name" : "elasticsearch_master",
      "transport_address" : "172.19.0.2:930

In [27]:
!curl -s -XPUT 'localhost:9200/wazuh-alerts-*/_settings' -H 'Content-Type: application/json' -d '{ "index": { "number_of_replicas": "0" } }'

In [29]:
!curl -s -XPUT localhost:9200/_cluster/settings -H "Content-Type: application/json" -d '{ "persistent": { "cluster.max_shards_per_node": "3000" } }'

{"error":{"root_cause":[{"type":"json_parse_exception","reason":"Unexpected character (''' (code 39)): expected a valid value (JSON String, Number, Array, Object or token 'null', 'true' or 'false')\n at [Source: (org.elasticsearch.common.io.stream.ByteBufferStreamInput); line: 1, column: 2]"}],"type":"json_parse_exception","reason":"Unexpected character (''' (code 39)): expected a valid value (JSON String, Number, Array, Object or token 'null', 'true' or 'false')\n at [Source: (org.elasticsearch.common.io.stream.ByteBufferStreamInput); line: 1, column: 2]"},"status":400}


In [33]:
!curl -s -XGET localhost:9200/_cluster/stats?filter_path=indices.shards.total

{"indices":{"shards":{"total":1502}}}
